In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('Human Resources.csv')

DATA AUDIT

In [3]:
#information about the dataset
print("Human Resources Dataset:")
print("\n\nHead about the dataset:")
print(data.head())
print("----------------------------------------")
print("\n\nShape about the dataset:")
print(data.shape)
print("----------------------------------------")
print("\n\nColumns about the dataset:")
print(data.columns)
print("\n\nChecking for missing values in each column:")
print(data.isnull().sum())
print("----------------------------------------")
print("\n\nNumber of duplicated rows in the dataset:")
print(data.duplicated().sum())
print("----------------------------------------")
print("\n\nInformation about the dataset:")
print(data.info())
print("----------------------------------------")
print("\n\nStatistical summary of the dataset:")
print(data.describe())
print("----------------------------------------")
print("\n\nUnique values in each column:")
for column in data.columns:
    print(f"{column}: {data[column].nunique()}")


Human Resources Dataset:


Head about the dataset:
           id first_name    last_name  birthdate  gender  \
0  00-0037846      Kimmy   Walczynski   6/4/1991    Male   
1  00-0041533   Ignatius    Springett  6/29/1984    Male   
2  00-0045747     Corbie  Bittlestone  7/29/1989    Male   
3  00-0055274       Baxy       Matton  9/14/1982  Female   
4  00-0076100    Terrell         Suff  4/11/1994  Female   

                        race            department  \
0         Hispanic or Latino           Engineering   
1                      White  Business Development   
2  Black or African American                 Sales   
3                      White              Services   
4          Two or More Races    Product Management   

                     jobtitle      location   hire_date  \
0        Programmer Analyst I  Headquarters   1/20/2002   
1            Business Analyst  Headquarters    4/8/2019   
2  Solutions Engineer Manager  Headquarters  10/12/2010   
3                Service Te

DATA CLEANING

In [4]:
#removing rows which are having invalid ID
data = data[data['id'].notna()].copy()

In [5]:
#standarizing date columns
data['birthdate'] = pd.to_datetime(data['birthdate'], errors='coerce')
data['hire_date'] = pd.to_datetime(data['hire_date'], errors='coerce')

data['termdate'] = (
    pd.to_datetime(data['termdate'], errors='coerce')
      .dt.tz_localize(None)
)


In [8]:
#fixing categorical columns
cat_cols = [
    'gender', 'race', 'department', 'jobtitle',
    'location', 'location_city', 'location_state'
]

for col in cat_cols:
    data[col] = data[col].astype('category')


FEATURE ENGINEERING

In [9]:
#age 
today = pd.Timestamp.today()
data['age'] = (today - data['birthdate']).dt.days // 365

In [10]:
#tenure
data['tenure_years'] = (
    (data['termdate'].fillna(today) - data['hire_date'])
    .dt.days / 365
).round(1)


In [11]:
#employee status
data['employment_status'] = data['termdate'].apply(
    lambda x: 'Terminated' if pd.notna(x) else 'Active'
)


FACT TABLE: STAR SCHEMA

In [12]:
fact_employees = data[[
    'id',
    'hire_date',
    'termdate',
    'tenure_years',
    'age',
    'employment_status',
    'department',
    'location'
]].copy()


In [13]:
fact_employees['employee_key'] = (
    fact_employees['id']
    .astype('category')
    .cat.codes
)

fact_employees = fact_employees[[
    'employee_key',
    'id',
    'hire_date',
    'termdate',
    'tenure_years',
    'age',
    'employment_status',
    'department',
    'location'
]]

DIM TABLE: STAR SCHEMA

In [14]:
dim_department = (
    data[['department']]
    .dropna()
    .drop_duplicates()
    .reset_index(drop=True)
)

dim_department['department_key'] = dim_department.index + 1

dim_department = dim_department[[
    'department_key',
    'department'
]]


In [15]:
fact_employees = fact_employees.merge(
    dim_department,
    on='department',
    how='left'
)

fact_employees.drop(columns=['department'], inplace=True)

In [16]:
dim_location = (
    data[['location', 'location_city', 'location_state']]
    .dropna(subset=['location'])
    .drop_duplicates()
    .reset_index(drop=True)
)

dim_location['location_key'] = dim_location.index + 1

dim_location = dim_location[[
    'location_key',
    'location',
    'location_city',
    'location_state'
]]


In [17]:
fact_employees = fact_employees.merge(
    dim_location[['location_key', 'location']],
    on='location',
    how='left'
)

fact_employees.drop(columns=['location'], inplace=True)


In [18]:
min_date = data['hire_date'].min()
max_date = pd.Timestamp.today()
dim_date = pd.DataFrame({
    'date': pd.date_range(start=min_date, end=max_date)
})

dim_date['date_key'] = dim_date['date'].dt.strftime('%Y%m%d').astype(int)
dim_date['year'] = dim_date['date'].dt.year
dim_date['quarter'] = dim_date['date'].dt.quarter
dim_date['month'] = dim_date['date'].dt.month
dim_date['month_name'] = dim_date['date'].dt.strftime('%B')
dim_date['year_month'] = dim_date['date'].dt.strftime('%Y-%m')

In [19]:
#exporting
fact_employees.to_csv('FactEmployees.csv', index=False)
dim_department.to_csv('DimDepartment.csv', index=False)
dim_location.to_csv('DimLocation.csv', index=False)
dim_date.to_csv('DimDate.csv', index=False)